## Process 19F NMR spectra
#### You need:
- The data from the spectrometer
#### You will:
1) Import the 1H Bruker data
2) Use the 1H spectrum to reference
3) Import the 19F Bruker data
    - Optionally sum multiple spectra into a single spectrum
4) Baseline the spectrum by fitting a fourth-order polynomial
5) Fit a Lorentzian function to each peak
6) Produce a plot

Firstly you need to make sure that Python and the required libraries are installed.

To install Python (latest version): 
- Mac https://www.python.org/downloads/macos/
- Windows https://www.python.org/downloads/windows/

To install the libraries type this in your terminal:
`pip install numpy matplotlib scipy nmrglue ipywidgets`

Now you can import the libraries by clicking the arrow to the left of the cell below

In [2]:
# Import libraries
import nmrglue as ng # This is for processing the Bruker data
import matplotlib.pyplot as plt # This is for making the plots
import numpy as np # This is for polynomial fitting
from scipy.optimize import curve_fit # This is for Lorentzian fitting
import ipywidgets as widgets # This is for the interactive parts
from ipywidgets import interact
print("Import OK")

Import OK


If the libraries imported OK, you can proceed.

Now you will import your 1H spectrum for referencing. 
- Locate the NMR data on your computer
- The first experiment, folder `1`, contains the acquisition data
- The data is stored in a file called `fid` with no extension
- Provide the path to folder `1` in the cell below. Do not delete the quotation marks
- Run the cell

To get the path, right-click the folder in Finder/File explorer, then hold Option/Shift and click `Copy pathname`

First step is to reference the 1H spectrum by aligning the DSS peak to 0 ppm. The amount you reference the 1H spectrum by is multiplied by 0.94 to reference the 19F spectrum (ratio of gyromagnetic ratios).

Set the path below to folder 2 of your data from the spectrometer and run the cell:

In [3]:
h_path = "./test_data/2"

Before referencing the spectrum, you need to Fourier transform (automatic) and phase correct. Adjust the slider until the peaks are symmetrical.

In [ ]:
# Run this cell to read the data

@interact(
    p0=widgets.FloatSlider(min=-70, max=70, step=0.5, value=-25, description="p0", readout_format=".1f"),
    offset=widgets.FloatSlider(min=-0.1, max=0.1, step=0.001, value=0.0, description="Offset", readout_format=".3f")
)
def proc_1h(p0=0, offset=0.0):
    p0_in = p0
    dic, data = ng.bruker.read(h_path)
    data = ng.bruker.remove_digital_filter(dic, data)

    data = ng.proc_base.zf_size(data, 32768)    # zero fill to 32768 points
    data = ng.proc_base.fft(data)               # Fourier transform
    data = ng.proc_base.ps(data, p0=p0_in)      # phase correction
    data = ng.proc_base.di(data)                # discard the imaginaries
    data = ng.proc_base.rev(data) 

    udic = ng.bruker.guess_udic(dic, data)
    uc = ng.fileiobase.uc_from_udic(udic)
    ppm_scale = uc.ppm_scale()
    ppm_scale = uc.ppm_scale() - offset
    plt.figure(figsize=(5, 5))
    plt.plot(ppm_scale, data, color='b')
    plt.xlabel("Chemical Shift (ppm)")
    plt.ylabel("Intensity")
    plt.title("1H NMR Spectrum")
    plt.xlim(-1, 2)
    plt.axvline(x=0, color='lightgrey', linestyle='--', linewidth=0.5)
    y_min, y_max = min(data), max(data)
    zoom_factor = 2  # Adjust this to zoom in/out
    plt.ylim(y_min / zoom_factor, y_max / zoom_factor)
    plt.gca().invert_xaxis()



interactive(children=(FloatSlider(value=-25.0, description='p0', max=70.0, min=-70.0, readout_format='.1f', st…

Enter the offset value here and run the cell:

In [9]:
ref_offset = -0.068

Now you will load the 19F data. Enter the numbers of the first and last experiments below then run the cell. If you only have one experiment set the same number for both.

In [ ]:
first_exp = 3
last_exp = 30